In [1]:
#Load google vectors through bin file

from gensim.models.word2vec import Word2Vec

model = Word2Vec.load_word2vec_format('Base/GoogleNews-vectors-negative300.bin', binary=True)


In [91]:
import numpy as np

with open('Base/food_words.txt', 'rb') as infile:
    food_words = infile.readlines()
    
with open('Base/sports_words.txt', 'rb') as infile:
    sport_words = infile.readlines()
    
with open('Base/weather_words.txt', 'rb') as infile:
    weather_words = infile.readlines()
    
uwords =[]

def getWordVecs(words):
    vecs = []
    for word in words:
        word = word.replace("\r\n", "")
        try:
            vecs.append(model[word])
            uwords.append(word)
        except KeyError:
            continue
    #print len(vecs)
    #vecs = np.concatenate(vecs)
    #return vecs
    return np.array(vecs, dtype='float') #TSNE expects float type values

food_vecs = getWordVecs(food_words)
#sports_vecs = getWordVecs(sport_words)
#weather_vecs = getWordVecs(weather_words)

In [92]:
len(uwords)
#palavras compostas quebrar e somar os dois vetores

468

In [ ]:
#Just for using less memory (Won't need load bin file)
#write whole vectors in a text file
with open('all_vectors_out.txt', 'w') as f:
    for row in food_vecs:
        f.write(str(row))
        f.write('\n')
    for row in sports_vecs:
        f.write(str(row))
        f.write('\n')
    for row in weather_vecs:
        f.write(str(row))
        f.write('\n')        
    f.close()
    
#Load text file
vectors = []
with open('all_vectors_out.txt', 'r') as a:
    all_vectors = [float(x) for x in a.readlines()]

In [98]:
from scipy.spatial.distance import *
import math

#Duas chaves tranforma em matriz 1X1
clusterVec = [list(food_vecs[0])]         # tracks sum of vectors in a cluster
clusterIdx = [[0]]         # array of index arrays. e.g. [[1, 3, 5], [2, 4, 6]]
ncluster = 1
# probablity to create a new cluster vectors
# is not strongly "similar" to any existing table
pnew = 1.0/ (1+ ncluster) 
N = len(food_vecs)
rands = np.random.rand(N)         # N rand variables sampled from U(0, 1)
 
for i in range(1,N):
    maxSim = -1
    maxIdx = 0
    #List para transformar em matriz normal
    v = list(food_vecs[i])
    for j in range(ncluster):
        sim = 1- cosine(clusterVec[j], v)
        if math.isnan(sim):
            sim = 0
        if sim > maxSim:
            maxIdx = j
            maxSim = sim
    if maxSim < 2*pnew:                
        if rands[i] < pnew:
            clusterVec.append([])
            clusterIdx.append([])
            ncluster += 1
            clusterVec[ncluster-1] = v
            clusterIdx[ncluster-1] = [i]
            pnew = 1.0 / (1 + ncluster)
        else:
            clusterVec[maxIdx] = map(sum, zip(clusterVec[maxIdx], v))
            clusterIdx[maxIdx].append(i)

    else: 
        clusterVec[maxIdx] = map(sum, zip(clusterVec[maxIdx], v))
        clusterIdx[maxIdx].append(i)
    
print [[uwords[i] for i in c] for c in clusterIdx]

        #clusterIdx.insert(maxIdx, i)
    #if ncluster == 0:
    #    ncluster =+ 1

[['almond', 'anise', 'apple', 'apricot', 'artichoke', 'asparagus', 'avocado', 'banana', 'barley', 'basil', 'beans', 'beet', 'berry', 'bitter', 'blackberry', 'blueberry', 'boysenberry', 'bran', 'breadfruit', 'broccoli', 'buckwheat', 'cantaloupe', 'capers', 'carrot', 'cashew', 'cassava', 'cauliflower', 'caviar', 'celery', 'cereal', 'chard', 'cherry', 'chives', 'cilantro', 'cinnamon', 'citron', 'citrus', 'clam', 'cloves', 'coconut', 'cod', 'corn', 'crab', 'cranberry', 'cream', 'cucumber', 'currants', 'daikon', 'dairy', 'dates', 'dill', 'dragonfruit', 'dried', 'dry', 'durian', 'edible', 'eggplant', 'elderberry', 'endive', 'fennel', 'fig', 'fish', 'flax', 'flour', 'foodstuffs', 'fruit', 'garlic', 'ginger', 'grain', 'grape', 'grapefruit', 'grated', 'greens', 'guava', 'herbs', 'halibut', 'hazelnut', 'herbs', 'honey', 'honeydew', 'horseradish', 'jackfruit', 'jalapeno', 'jicama', 'julienne', 'juice', 'kale', 'kiwi', 'kohlrabi', 'kumquat', 'lamb', 'legumes', 'lemon', 'lentils', 'lettuce', 'licor

[['almond',
  'anise',
  'apple',
  'apricot',
  'artichoke',
  'asparagus',
  'avocado',
  'banana',
  'barley',
  'basil',
  'batter',
  'beans',
  'beet',
  'berry',
  'bitter',
  'blackberry',
  'blueberry',
  'boysenberry',
  'bran',
  'breadfruit',
  'broccoli',
  'buckwheat',
  'cantaloupe',
  'capers',
  'caramel',
  'carrot',
  'cashew',
  'cassava',
  'cauliflower',
  'celery',
  'cereal',
  'chard',
  'cheddar',
  'cherry',
  'chives',
  'chutney',
  'cilantro',
  'cinnamon',
  'citron',
  'citrus',
  'clam',
  'cloves',
  'coconut',
  'cod',
  'corn',
  'crab',
  'cranberry',
  'cream',
  'cucumber',
  'currants',
  'daikon',
  'dairy',
  'dill',
  'dragonfruit',
  'dried',
  'dry',
  'durian',
  'edible',
  'eggplant',
  'elderberry',
  'endive',
  'fennel',
  'fig',
  'fish',
  'flax',
  'flour',
  'foodstuffs',
  'fruit',
  'garlic',
  'gelatin',
  'ginger',
  'grain',
  'grape',
  'grapefruit',
  'grated',
  'greens',
  'guava',
  'herbs',
  'halibut',
  'hazelnut',
  '

In [ ]:
#vetor = np.cumsum(food_vecs) #how does cumsum make?
#either I sum all of them or I sum them apllying PCA

m = np.array([np.mean(food_vecs,axis=0), np.mean(sports_vecs,axis=0), np.mean(weather_vecs,axis=0)])

#print food_vecs.shape
print np.mean(m,axis=0).shape
#import gensim
#print gensim.__version__

In [ ]:
from scipy.spatial.distance import cdist

#idx = np.argmin(cdist(m,[model['acorn']],'cosine'),axis=0)
cdist(m,[model['eat']],'cosine')



In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
vetor_res = pca.fit_transform(food_vecs)